# Задание на coursera #

## Логистическая регрессия ##

Цель:
* работать с логистической регрессией
* реализовывать градиентный спуск для ее настройки
* использовать регуляризацию

**Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.**

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from typing import Tuple


In [20]:
name = ['Y', 'x1','x2']
data = pd.read_csv('data-logistic .csv',names=name)
y = data.Y
X = data.drop(['Y'], axis=1)
X

,x1,x2
0,-0.663827,-0.138526
1,1.994596,2.468025
2,-1.247395,0.749425
3,2.309374,1.899836
4,0.849143,2.407750
...,...,...
200,4.245176,3.053931
201,2.437935,1.357804
202,-1.876638,1.533398
203,-6.824446,-13.934211


**Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).**

![Альтернативный текст](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/hrIFu8eQEeWoXw5ZvVCRuw_1a61e227f5a1bc7fc0354df00fa70781_logderiv.png?expiry=1601683200000&hmac=r1sKPDFEv6RhKxRsIszgySgL6k3kXjqh284ihRDu-cQ)

In [30]:
def calc_w1(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X['x1'][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w1*X['x1'][i] + w2*X['x2'][i]))))
    return w1 + (k * (1.0 / l) * S) - k * C * w1


![Альтернативный текст](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/kb-c5MeQEeW7RxKvROGwrw_48f340a33cb38e1de9317b7df29d1cda_logderiv2.png?expiry=1601683200000&hmac=JHEJohFZ65UVKpxpghZwW-PBiqCEzkmN--fY0ikxG1E)

In [31]:
def calc_w2(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X['x2'][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w1*X['x1'][i] + w2*X['x2'][i]))))

    return w2 + (k * (1.0 / l) * S) - k * C * w2

In [32]:
def gradient_descent(X: pd.DataFrame, y: pd.Series, w1: float=0.0, w2: float=0.0,
         k: float=0.1, C: float=0.0, precision: float=1e-5, max_iter: int=10000) -> Tuple[float, float]:
    for i in range(max_iter):
        w1_prev, w2_prev = w1, w2
        w1, w2 = calc_w1(X, y, w1, w2, k, C), calc_w2(X, y, w1, w2, k, C)
        if np.sqrt((w1_prev - w1) ** 2 + (w2_prev - w2) ** 2) <= precision:
            break

    return w1, w2

**Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5)**

In [34]:
w1, w2 = gradient_descent(X, y)
w1_reg, w2_reg = gradient_descent(X, y, C=10.0)

**5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании?**

*Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией:
a(x) = 1 / (1 + exp(-w1 * x1 - w2 * x2)).*

In [42]:
def a(X: pd.DataFrame, w1: float, w2: float) -> pd.Series:
    return 1.0 / (1.0 + np.exp(-w1 * X['x1'] - w2 * X['x2']))
y_proba = a(X, w1, w2)
y_proba_reg = a(X, w1_reg, w2_reg)

auc = roc_auc_score(y, y_proba)
auc_reg = roc_auc_score(y, y_proba_reg)
print(round(auc, 3))
print(round(auc_reg, 3))

0.927
0.936
